In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob

"""Change to the data folder"""
new_path = "./new_train/new_train"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [7]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)

In [90]:
val_dataset[200000]

{'city': 'MIA',
 'lane': array([[ 826.7133 , 2024.8704 ,    0.     ],
        [ 826.7629 , 2023.2972 ,    0.     ],
        [ 826.81244, 2021.7241 ,    0.     ],
        ...,
        [ 777.62354, 1977.201  ,    0.     ],
        [ 778.81177, 1977.2445 ,    0.     ],
        [ 780.     , 1977.2878 ,    0.     ]], dtype=float32),
 'lane_norm': array([[ 0.04956238, -1.5731499 ,  0.        ],
        [ 0.04956238, -1.5731499 ,  0.        ],
        [ 0.04956238, -1.5731499 ,  0.        ],
        ...,
        [ 1.1882372 ,  0.04340362,  0.        ],
        [ 1.1882372 ,  0.04340362,  0.        ],
        [ 1.1882372 ,  0.04340362,  0.        ]], dtype=float32),
 'scene_idx': 94650,
 'agent_id': '00000000-0000-0000-0000-000000088787',
 'car_mask': array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [

### Create a loader to enable batch processing

In [8]:
batch_sz = 4

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    inp = torch.LongTensor(inp)
    out = torch.LongTensor(out)
    return [inp, out]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

### Visualize the batch of sequences

In [12]:
import matplotlib.pyplot as plt
import random

agent_id = 0

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])

In [87]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class NN(nn.Module):
    """This class defines your deep learning model that extends a Module class
      The constructor of your class defines the layers of the model. 
      The forward() function defines how to forward propagate 
      input through the defined layers of the model.

    """
    def __init__(self):
        super(NN, self).__init__()
#         self.flatten = nn.Flatten()
        self.LSTM_stack = nn.Sequential(
            nn.LSTM(input_size=76, hidden_size=30, num_layers=1, batch_first=True),
        )

    def forward(self, x):
        x = torch.reshape(x, (4,60,76))
        x = x.float()
        print(x.shape)
        out, (hn, cn) = self.LSTM_stack(x)
        print(out.shape)
        return out

In [84]:
print(NN())

NN(
  (LSTM_stack): Sequential(
    (0): LSTM(76, 30, batch_first=True)
  )
)


In [85]:
from tqdm import tqdm_notebook as tqdm
def train(model, device, train_loader, optimizer, epoch, log_interval=10000):
    model.train()
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    for i_batch, sample_batch in enumerate(train_loader):
        inp, out = sample_batch
        """TODO:
          Deep learning model
          training routine
        """
        data, target = inp.to(device), out.to(device)
        optimizer.zero_grad()
        output = model(data)
        print(output.shape)
        print(target.shape)
        loss = F.nll_loss(output, target.long())
        loss.backward()
        optimizer.step()
        counter += 1
        iterator.set_postfix(loss=(loss.item()*data.size(0) / (counter * train_loader.batch_size)))
#         print(out.shape)
#         show_sample_batch(sample_batch, agent_id)
#         break

In [88]:
device = "cuda"
model = NN().to(device) #using gpu here
optimizer = optim.Adagrad(model.parameters(), lr=0.01, lr_decay=0, weight_decay=0, initial_accumulator_value=0)
num_epoch = 2

for epoch in range(1, num_epoch + 1):
        train(model, device, val_loader, optimizer, epoch)
#         test(model, device, test_loader)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


torch.Size([4, 60, 76])
torch.Size([4, 60, 30])
torch.Size([4, 60, 30])
torch.Size([4, 60, 30, 4])


ValueError: Expected target size (4, 30), got torch.Size([4, 60, 30, 4])